In [1]:
# LightGBM_inference.py

import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
from dateutil.relativedelta import relativedelta
import time
import pickle
import sys
from pathlib import Path
root_dir = Path(__file__).resolve().parent.parent
sys.path.append(str(root_dir))
from utils.data_preparation import preprocess_data, split_data, mapping
from processing.custom_metrics import nash_sutcliffe, kling_gupta
from models.best_params.LightGBM_params import HYPERPARAMETERS
from tqdm import tqdm  # progress bar

NameError: name '__file__' is not defined

In [ ]:
with open('plotting_data/LOS_DAMM_Anomaly1.pkl', 'rb') as f:
    plotting_data = pickle.load(f)

# Extract data for plotting
X_all = plotting_data['X_all']
dates = plotting_data['dates']
actual_y = plotting_data['actual_y']
predictions = plotting_data['predictions']
split_idx = plotting_data['split_idx']
#coefficients = plotting_data['coefficients']
RMSE = plotting_data['RMSE']
y_test = actual_y.iloc[split_idx:]
X_test = X_all.iloc[split_idx:]
y_train = actual_y.iloc[:split_idx]
train_predictions = predictions[:split_idx]
test_predictions = predictions[split_idx:]
test_dates = pd.to_datetime(dates.iloc[split_idx:])
test_residuals = y_test - test_predictions
train_residuals = np.abs(y_train - train_predictions)

confidence = 0.50
alpha = (1 - confidence) / 2
lower_bound = np.quantile(test_residuals, alpha)
upper_bound = np.quantile(test_residuals, 1 - alpha)
print(lower_bound, upper_bound)
def coverage_fraction(y, y_low, y_high):
    return np.mean(np.logical_and(y >= y_low, y <= y_high))
print(coverage_fraction(y_test, test_predictions + lower_bound, test_predictions + upper_bound))

anomaly_start_date = pd.to_datetime('2023-01-12')
anomaly_start_index_in_test = np.where(test_dates >= anomaly_start_date)[0][0]
transition_period = len(test_dates) - anomaly_start_index_in_test
final_factor = -0.0015

test_anomaly = np.copy(y_test)

for day in range(transition_period):
        if anomaly_start_index_in_test + day < len(test_dates):
            factor = (1 - np.abs(final_factor) * day / transition_period)
            # positive or negative anomaly based on the sign of final_factor
            test_anomaly[anomaly_start_index_in_test + day] *= factor if final_factor < 0 else 1 / factor


"""
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

beta = 2
F2 = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall) if (beta**2 * precision + recall) > 0 else 0

# Print results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F2 Score: {F2:.2f}")
"""
def cusum(y, k, h, index):
    S = np.zeros_like(y)
    alarms = []
    S[0] = 0
    alarm_triggered = False  # To track if an alarm was triggered

    for i in range(index, len(y)):
        if not alarm_triggered:
            S[i] = max(0, S[i-1] + y[i] - k)
            if S[i] > h:
                alarms.append(i)
                alarm_triggered = True  # Stop further alarms

    return S, alarms

# Assuming test_residuals is an array of residuals from your model
positive_residuals = test_residuals[test_residuals > 0]
negative_residuals = np.abs(test_residuals[test_residuals < 0])  # Take absolute for ease of quantile calculation

k_percentile = 0.80  # 75th percentile for initial significant change detection
h_percentile = 0.95  # 95th percentile for confirming a sustained shift

k_upper = np.quantile(positive_residuals, k_percentile)
h_upper = np.quantile(positive_residuals, h_percentile)

k_lower = np.quantile(negative_residuals, k_percentile)
h_lower = np.quantile(negative_residuals, h_percentile)

print(k_lower, h_lower)
print(k_upper, h_upper)

deviations_upper = test_anomaly - (test_predictions + upper_bound)
deviations_lower = (test_predictions + lower_bound) - test_anomaly

S_upper, alarms_upper = cusum(deviations_upper.clip(min=0), k_upper, h_upper, anomaly_start_index_in_test)  
S_lower, alarms_lower = cusum(deviations_lower.clip(min=0), k_lower, h_lower, anomaly_start_index_in_test)

print(alarms_upper)
print(alarms_lower)
print(test_predictions[alarms_lower])

plt.clf()
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1, 1]}, sharex=False)  

# Data and predictions
#for alarm in alarms_upper:
    #ax1.axvline(x=test_dates.iloc[alarm], color='purple', linestyle='--', label='Alarm Upper', linewidth=1)
#for alarm in alarms_lower:
    #ax1.axvline(x=test_dates.iloc[alarm], color='purple', linestyle='--', label='Alarm Lower', linewidth=1)
#ax1.scatter(test_dates, y_test, color='blue', s=1, label='Actual data')
#ax1.plot(test_dates, y_test, color='blue', linewidth=0.5, alpha=0.3)
n = 4947
ax1.plot(test_dates, test_anomaly, color='orange', linewidth=1, label='Anomaly')
#ax1.plot(test_dates[:n], test_predictions[:n], color='red', linewidth=1, label='Prediction')
ax1.plot(test_dates, (test_predictions + upper_bound), 'k-')
ax1.plot(test_dates, (test_predictions + lower_bound), 'k-')
#ax1.axvline(x=test_dates.iloc[5960], color='purple', linestyle='--', label='Alarm at 70% CI')
#ax1.axvline(x=test_dates.iloc[3870], color='purple', linestyle='--', label='Alarm at 68% CI')
#ax1.axvline(x=dates.iloc[split_idx], color='green', linestyle='--')
ax1.axvline(x=test_dates.iloc[anomaly_start_index_in_test], color='orange', linestyle='--', label='Start of anomaly')
ax1.set_ylabel('DPM_05 (m)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
#ax1.legend(loc='upper right')
ax1.grid()
ax1.tick_params(axis='x', rotation=45)


# CUSUM plots
ax2.plot(test_dates, S_upper, color='purple', label='CUSUM Upper')
ax2.axhline(y=h_upper, color='black', linestyle='--', label=f'h_upper={h_upper:.2f}')
ax2.set_ylabel('CUSUM Upper', color='purple')
ax2.tick_params(axis='y', labelcolor='purple')
ax2.legend(loc='upper left')

ax3.plot(test_dates, S_lower, color='purple', label='CUSUM Lower')
ax3.axhline(y=h_lower, color='black', linestyle='--', label=f'h_lower={h_lower:.2f}')
ax3.set_ylabel('CUSUM Lower', color='purple')
ax3.tick_params(axis='y', labelcolor='purple')
ax3.legend(loc='upper left')

# Formatting
plt.xticks(rotation=45)  
fig.tight_layout()  

# Text
fig.subplots_adjust(right=0.75)
rmse_text = f'RMSE: {RMSE:.3f}'
#coeff_text = 'Coefficients:\n' + '\n'.join([f'{term}: {coeff:.2e}' for term, coeff in coefficients.items()])
fig.text(0.77, 0.9, rmse_text, verticalalignment='top')
#fig.text(0.77, 0.8, coeff_text, verticalalignment='top')

plt.show()
